# Rely on LLM's to extract info from docs

Will save them in JSON for future reuse.

In [1]:
import os
import boto3
import json
import dateparser
from glob import glob
from rag.basic_retrieval import file_id
from cachier import cachier

from Templates.ibis_aws_summary_template import TEMPLATE as IBIS_SUMMARY_TEMPLATE
from Templates.aws_basic_info_template import TEMPLATE as BASIC_TEMPLATE
from Templates.aws_templates_common import build_aws_template

from loading_utils import get_initial_pages

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
AWS_REGION_NAME = 'us-west-2'
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html
bedrock = boto3.client(
    service_name='bedrock-runtime',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=AWS_REGION_NAME
)

In [3]:
# trouble-shooting: use a different client with service_name 'bedrock', not 'bedrock-runtime'
# https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock.html

# client = boto3.client(
#     service_name='bedrock',
#     aws_access_key_id=aws_access_key_id,
#     aws_secret_access_key=aws_secret_access_key,
#     region_name=AWS_REGION_NAME
# )

# summ = client.list_foundation_models()['modelSummaries']
# [model for model in summ if 'Sonnet' in model['modelName']]

In [4]:
# [m for m in dir(bedrock) if not m.startswith('_')]
# help(bedrock.converse)

In [5]:
def get_raw_pdf_part(filename: str) -> dict:
    """This works best and parses quickly."""
    with open(filename, 'rb') as f:
        content = f.read()
        return {
            "document": {
                "format": "pdf",
                "name": 'document',
                "source": {
                    "bytes": content
                }
            }
        }


def response_to_template(filename: str, template: dict) -> dict:

    prompt = """
    You are an expert in extracting market and financial data from documents.
    Extract essential data from text in the enclosed document.

    Return the result in JSON format. Do not use non-JSON tags such as <property> or <UNKNOWN>.
    Use only simple keys with units, such as "historical_revenue_growth_percentage" or "establishments_count" or "revenue_dollars".
    """

    initial_message = {
        "role": "user",
        "content": [
            {
                "text": prompt,
            },
        ],
    }

    initial_message['content'].append(get_raw_pdf_part(filename))

    tool_list = [{
        "toolSpec": template
    }]
    response = bedrock.converse(
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",
        # modelId="meta.llama3-1-405b-instruct-v1:0",
        messages=[initial_message],
        inferenceConfig={
            "temperature": 0
        },
        toolConfig={
            "tools": tool_list,
            "toolChoice": {
                "tool": {
                    "name": "info_extract"
                }
            }
        }
    )
    core_response = response['output']['message']['content'][0]['toolUse']['input']
    if 'properties' in core_response:
        core_response: dict = core_response['properties']
    for k, v in core_response.items():
        if isinstance(v, str) and v[0] in '{[' and v[-1] in ']}':
            try:
                core_response[k] = json.loads(v)
            except Exception:
                pass

    return core_response

In [6]:
# def filename_template_hash(filename: str, template: dict):
def filename_template_hash(*args):
    # TODO: figure out why it works this way
    filename = args[1]['filename']
    template = args[1]['template']
    return (file_id(filename), template['name'])


# @cachier(hash_func=filename_template_hash)
def info_from_doc_template(filename: str, template: dict) -> dict:
    """Populate the separate templates and merge the result."""

    template_parts = template['data']
    full_templates = build_aws_template(template_parts)
    results = [response_to_template(filename, part) for part in full_templates]

    total = {}
    for result in results:
        total.update(result)

    return total

In [7]:
def extract_basic_info(filename: str) -> dict:
    """Extract basic info based on the template and initial part of the file."""
    pages_filename = get_initial_pages(filename, pmax=10)
    result = info_from_doc_template(filename=pages_filename, template=BASIC_TEMPLATE)
    os.remove(pages_filename)
    return result

In [8]:
# filenames = glob('IndustrySource/Misc/*')

filenames = [
    'IndustrySource/Misc/51 Information in the US Industry Report copy.pdf',
    'IndustrySource/Misc/United_States_Healthcare_Repor copy.pdf'
]

In [9]:
out_basic = [extract_basic_info(filename) for filename in filenames[:5]]
out_basic

[{'title': 'Information in the US',
  'source': 'IBISWorld',
  'last_updated': 'Jun 2024'},
 {'title': 'United States Healthcare Report',
  'source': 'BMI',
  'last_updated': 'April 2024'}]

In [10]:
def ibis_industry_summary(filename: str) -> dict:
    """Extract full summary from the PDF file."""
    return info_from_doc_template(filename=filename, template=IBIS_SUMMARY_TEMPLATE)

In [11]:
out_full = [ibis_industry_summary(filename) for filename in filenames[:5]]
out_full

[{'executive_summary': 'The Information sector creates and distributes media content to US consumers and businesses. The sector responds to trends in household formation, advertising expenditure, consumer incomes and spending habits. Through 2024, sector revenue will expand at a CAGR of 2.7% to reach $2.4 trillion. Although consumer demand for media is steady, revenue flows are uneven due to technology trends. Substantial expansion has stemmed from new consumer devices and online publishing/data processing, at the expense of traditional subsectors like print publishing. Through 2029, the sector will continue expanding in line with the economy at a CAGR of 2.2% to reach $2.7 trillion.',
  'key_statistics': {'historical_revenue_growth_percentage': 2.7,
   'profit_margins_percentage': 12.7,
   'projected_revenue_growth_percentage': 2.2,
   'revenue_dollars': 2423898700000},
  'current_performance': 'The Information sector creates and distributes media content to US consumers and businesse

In [12]:
def parse_save_file(filename: str):
    """Extract info and save it."""
    ibis_summary = ibis_industry_summary(filename)
    last_updated = dateparser.parse(ibis_summary['last_updated']).isoformat()
    last_updated = last_updated.replace(' ', 'T') + '+00:00'

    out = {
        'title': f'{ibis_summary["industry_name"]} Industry Report',

        'category': 'Industry research',
        'subcategory': 'IBIS Industry at a Glance',
        'tags': ['test', 'industry report', 'IBIS'],
        
        'last_updated': last_updated,

        'summary': ibis_summary,
    }

    out_filename = filename.replace('Source', 'Summary').replace('.pdf', '.json')
    with open(out_filename, 'w') as f:
        json.dump(out, f, indent=2)

In [ ]:
filenames = [
    'IndustrySource/IBIS/Household Furniture Manufacturing in the US.pdf',
    'IndustrySource/IBIS/Furniture Stores in the US.pdf',
    "IndustrySource/IBIS/IT Consulting in the US.pdf",
]
for filename in filenames:
    parse_save_file(filename)

/tmp/ipykernel_45571/3654963537.py:20: ResourceWarning: unclosed file <_io.TextIOWrapper name='IndustrySummary/IBIS/Household Furniture Manufacturing in the US.json' mode='w' encoding='UTF-8'>
  json.dump(out, open(out_filename, 'w'), indent=2)
/tmp/ipykernel_45571/3654963537.py:20: ResourceWarning: unclosed file <_io.TextIOWrapper name='IndustrySummary/IBIS/Furniture Stores in the US.json' mode='w' encoding='UTF-8'>
  json.dump(out, open(out_filename, 'w'), indent=2)
